# 4 simulate scenario

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-guide-demo`.

In [ ]:
export PJ_ROOT="${HOME}/example-guide-demo"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-guide-demo
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-guide-demo`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## initialize the state of mobile_robot

### initialize `r_state` and `r_mode`

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/attrs?type=${MOBILE_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: eb4c2342-73db-11e9-bd2e-62712902dfb1
date: Sat, 11 May 2019 10:59:59 GMT
x-envoy-upstream-service-time: 5
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/attrs?type=${MOBILE_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "r_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: eb4c2342-73db-11e9-bd2e-62712902dfb1
date: Sat, 11 May 2019 10:59:59 GMT
x-envoy-upstream-service-time: 5
server: envoy
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=r_mode,r_state | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "r_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
```

## subscribe MQTT topics

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### start subscribing

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

## 1) a visitor tells his/her destination to `comm_robot`

### publish a MQTT message to emulate receiving the event from comm_robot

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${COMM_ROBOT_TYPE}/${COMM_ROBOT_ID}/attrs -m "${d}|move|dest01"

example)
```
Client mosq/U0TQvpEnc6aE7EkU7b sending CONNECT
Client mosq/U0TQvpEnc6aE7EkU7b received CONNACK (0)
Client mosq/U0TQvpEnc6aE7EkU7b sending PUBLISH (d0, q0, r0, m1, '/comm_robot/xperiahello_01/attrs', ... (48 bytes))
Client mosq/U0TQvpEnc6aE7EkU7b sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

**send `move|up` command to mobile_robot automatically.**

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/comm_robot/xperiahello_01/attrs', ... (48 bytes))
2019-05-12T11:23:08.1557627788+09:00|move|dest01
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmd', ... (18 bytes))
rosbot2_01@move|up
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=move_info,move_status | jq .

**the value of  `move_status` is `PENDING`**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:03:45.173Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:23:08.449Z"
      }
    }
  }
}
```

### publish a MQTT message to emulate receiving the command result from mobile_robot

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/cmdexe -m "${MOBILE_ROBOT_ID}@move|received up"

example)
```
Client mosq/akM9daa2g2b8qK2H0t sending CONNECT
Client mosq/akM9daa2g2b8qK2H0t received CONNACK (0)
Client mosq/akM9daa2g2b8qK2H0t sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/cmdexe', ... (27 bytes))
Client mosq/akM9daa2g2b8qK2H0t sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmdexe', ... (27 bytes))
rosbot2_01@move|received up
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=move_info,move_status | jq .

**the value of  `move_status` is `OK`**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:31:34.287Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:31:34.287Z"
      }
    }
  }
}
```

## 2) `mobile_robot` notifies that it has started moving

### publish a MQTT message to emulate receiving the internal state of mobile_robot

In [ ]:
mode="navi"
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|r_mode|${mode}"

example)
```
Client mosq/D9b8qv6drdo4i1Hqwx sending CONNECT
Client mosq/D9b8qv6drdo4i1Hqwx received CONNACK (0)
Client mosq/D9b8qv6drdo4i1Hqwx sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
Client mosq/D9b8qv6drdo4i1Hqwx sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
2019-05-12T11:38:36.1557628716+09:00|r_mode|navi
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=r_mode,r_state | jq .

**when the value of `r_mode` changes to "navi", the value of `r_state` changes to "guiding" automatically**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "r_mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T11:38:36.1557628716+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": "guiding",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:38:36.00Z"
      }
    }
  }
}
```

## 3) `mobile_robot` notifies its current position periodically

### publish a MQTT message to emulate receiving the position of mobile_robot

#### 1: At first, mobile_robot notifies its current position from near the initial point. But nothing happens because mobile_robot has not reached  the point to turn the LED on.

In [ ]:
x=0.05
y=0.01
theta=0.0
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|x|${x}|y|${y}|theta|${theta}"

example)
```
Client mosq/gK4hzUv1CdICXD3pqW sending CONNECT
Client mosq/gK4hzUv1CdICXD3pqW received CONNACK (0)
Client mosq/gK4hzUv1CdICXD3pqW sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
Client mosq/gK4hzUv1CdICXD3pqW sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
2019-05-12T11:48:44.1557629324+09:00|x|0.05|y|0.01|theta|0.0
```

#### 2: when the mobile_robot notifies its current position nearby the point to be turned LED on, `dest_led` receives the `action|on` command automatically.

In [ ]:
x=0.78
y=0.01
theta=0.0
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|x|${x}|y|${y}|theta|${theta}"

example)
```
Client mosq/AGcKEoaq6b3c494WMC sending CONNECT
Client mosq/AGcKEoaq6b3c494WMC received CONNACK (0)
Client mosq/AGcKEoaq6b3c494WMC sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
Client mosq/AGcKEoaq6b3c494WMC sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
2019-05-12T11:53:44.1557629624+09:00|x|0.78|y|0.01|theta|0.0
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmd', ... (16 bytes))
led_01@action|on
```

#### confirm dest_led entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/?attrs=action_info,action_status | jq .

**the value of  `action_status` is `PENDING`**

example)
```json
{
  "id": "led_01",
  "type": "led",
  "action_info": {
    "type": "commandResult",
    "value": "received on",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T10:46:10.650Z"
      }
    }
  },
  "action_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:53:43.713Z"
      }
    }
  }
}
```

### publish a MQTT message to emulate receiving the command result from dest_led

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DEST_LED_TYPE}/${DEST_LED_ID}/cmdexe -m "${DEST_LED_ID}@action|received on"

example)
```
Client mosq/ZNj0gIzdIo9Tjdrnm1 sending CONNECT
Client mosq/ZNj0gIzdIo9Tjdrnm1 received CONNACK (0)
Client mosq/ZNj0gIzdIo9Tjdrnm1 sending PUBLISH (d0, q0, r0, m1, '/led/led_01/cmdexe', ... (25 bytes))
Client mosq/ZNj0gIzdIo9Tjdrnm1 sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmdexe', ... (25 bytes))
led_01@action|received on
```

#### confirm dest_led entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/?attrs=action_info,action_status | jq .

**the value of  `action_status` is `OK`**

example)
```json
{
  "id": "led_01",
  "type": "led",
  "action_info": {
    "type": "commandResult",
    "value": "received on",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:59:47.818Z"
      }
    }
  },
  "action_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:59:47.818Z"
      }
    }
  }
}
```

## 4) `mobile_robot` notifies that it has finished moving

### publish a MQTT message to emulate receiving the internal state of mobile_robot

In [ ]:
mode="standby"
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|r_mode|${mode}"

example)
```
Client mosq/44dC7lLVwYgnr7raur sending CONNECT
Client mosq/44dC7lLVwYgnr7raur received CONNACK (0)
Client mosq/44dC7lLVwYgnr7raur sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (51 bytes))
Client mosq/44dC7lLVwYgnr7raur sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (51 bytes))
2019-05-12T12:08:09.1557630489+09:00|r_mode|standby
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=r_mode,r_state | jq .

**when the value of `r_mode` changes to "standby", the value of `r_state` changes to "suspending" automatically**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "r_mode": {
    "type": "string",
    "value": "standby",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T12:08:09.1557630489+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": "suspending",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:08:08.00Z"
      }
    }
  }
}
```

## 5) the visitor pushes the arrival button of `dest_button`

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DEST_BUTTON_TYPE}/${DEST_BUTTON_ID}/attrs -m "${d}|push|${d}"

example)
```
Client mosq/K09A0nnTo89nbMbjNl sending CONNECT
Client mosq/K09A0nnTo89nbMbjNl received CONNACK (0)
Client mosq/K09A0nnTo89nbMbjNl sending PUBLISH (d0, q0, r0, m1, '/button/button_01/attrs', ... (78 bytes))
Client mosq/K09A0nnTo89nbMbjNl sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

**send `move|return` command to mobile_robot automatically.**

**send `action|off` command to dest_led automatically.**

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/button/button_01/attrs', ... (78 bytes))
2019-05-12T12:14:21.1557630861+09:00|push|2019-05-12T12:14:21.1557630861+09:00
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmd', ... (22 bytes))
rosbot2_01@move|return
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmd', ... (17 bytes))
led_01@action|off
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=move_info,move_status | jq .

**the value of  `move_status` is `PENDING`**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:31:34.287Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:14:21.194Z"
      }
    }
  }
}
```

#### confirm dest_led entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/?attrs=action_info,action_status | jq .

**the value of  `action_status` is `PENDING`**

example)
```json
{
  "id": "led_01",
  "type": "led",
  "action_info": {
    "type": "commandResult",
    "value": "received on",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T02:59:47.818Z"
      }
    }
  },
  "action_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:14:21.247Z"
      }
    }
  }
}
```

### publish a MQTT message to emulate receiving the command result from mobile_robot

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/cmdexe -m "${MOBILE_ROBOT_ID}@move|received return"

example)
```
Client mosq/Gr0gThCfozLAMnfuTN sending CONNECT
Client mosq/Gr0gThCfozLAMnfuTN received CONNACK (0)
Client mosq/Gr0gThCfozLAMnfuTN sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/cmdexe', ... (31 bytes))
Client mosq/Gr0gThCfozLAMnfuTN sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmdexe', ... (31 bytes))
rosbot2_01@move|received return
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=move_info,move_status | jq .

**the value of  `move_status` is `OK`**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "move_info": {
    "type": "commandResult",
    "value": "received return",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:19:37.103Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:19:37.103Z"
      }
    }
  }
}
```

### publish a MQTT message to emulate receiving the command result from dest_led

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DEST_LED_TYPE}/${DEST_LED_ID}/cmdexe -m "${DEST_LED_ID}@action|received off"

example)
```
Client mosq/onEcaJqDUnvMMMNL1E sending CONNECT
Client mosq/onEcaJqDUnvMMMNL1E received CONNACK (0)
Client mosq/onEcaJqDUnvMMMNL1E sending PUBLISH (d0, q0, r0, m1, '/led/led_01/cmdexe', ... (26 bytes))
Client mosq/onEcaJqDUnvMMMNL1E sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmdexe', ... (26 bytes))
led_01@action|received off
```

#### confirm dest_led entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/?attrs=action_info,action_status | jq .

**the value of  `action_status` is `OK`**

example)
```json
{
  "id": "led_01",
  "type": "led",
  "action_info": {
    "type": "commandResult",
    "value": "received off",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:21:11.382Z"
      }
    }
  },
  "action_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:21:11.382Z"
      }
    }
  }
}
```

## 6) `mobile_robot` notifies that it has started moving

### publish a MQTT message to emulate receiving the internal state of mobile_robot

In [ ]:
mode="navi"
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|r_mode|${mode}"

example)
```
Client mosq/bCiirbgvUcURsiSx0u sending CONNECT
Client mosq/bCiirbgvUcURsiSx0u received CONNACK (0)
Client mosq/bCiirbgvUcURsiSx0u sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
Client mosq/bCiirbgvUcURsiSx0u sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
2019-05-12T12:26:06.1557631566+09:00|r_mode|navi
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=r_mode,r_state | jq .

**when the value of `r_mode` changes to "navi", the value of `r_state` changes to "returning" automatically**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "r_mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T12:26:06.1557631566+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": "returning",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:26:06.00Z"
      }
    }
  }
}
```

## 7) `mobile_robot` notifies its current position periodically

### publish a MQTT message to emulate receiving the position of mobile_robot

#### 1: At first, mobile_robot notifies its current position from near the goal point. But nothing happens because mobile_robot is returning to its initial point.

In [ ]:
x=1.00
y=0.01
theta=0.0
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|x|${x}|y|${y}|theta|${theta}"

example)
```
Client mosq/HMbUFeW9ltTzUCiQzX sending CONNECT
Client mosq/HMbUFeW9ltTzUCiQzX received CONNACK (0)
Client mosq/HMbUFeW9ltTzUCiQzX sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
Client mosq/HMbUFeW9ltTzUCiQzX sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
2019-05-12T12:29:15.1557631755+09:00|x|1.00|y|0.01|theta|0.0
```

#### 2: Although the mobile_robot notifies its current position nearby the point to be turned LED on, nothing happens because mobile_robot is returning to its initial point.

In [ ]:
x=0.78
y=0.01
theta=0.0
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|x|${x}|y|${y}|theta|${theta}"

example)
```
Client mosq/Kg51iLapFEWTsPMPdr sending CONNECT
Client mosq/Kg51iLapFEWTsPMPdr received CONNACK (0)
Client mosq/Kg51iLapFEWTsPMPdr sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
Client mosq/Kg51iLapFEWTsPMPdr sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (60 bytes))
2019-05-12T12:32:20.1557631940+09:00|x|0.78|y|0.01|theta|0.0
```

## 8) `mobile_robot` notifies that it has finished moving

### publish a MQTT message to emulate receiving the internal state of mobile_robot

In [ ]:
mode="standby"
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|r_mode|${mode}"

example)
```
Client mosq/oTv4IyhlHWv75kURTD sending CONNECT
Client mosq/oTv4IyhlHWv75kURTD received CONNACK (0)
Client mosq/oTv4IyhlHWv75kURTD sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (51 bytes))
Client mosq/oTv4IyhlHWv75kURTD sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TqE1AIm00sCAJwKzdD received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (51 bytes))
2019-05-12T12:34:24.1557632064+09:00|r_mode|standby
```

#### confirm the mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/?attrs=r_mode,r_state | jq .

**when the value of `r_mode` changes to "standby", the value of `r_state` changes to "waiting" automatically**

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "r_mode": {
    "type": "string",
    "value": "standby",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T12:34:24.1557632064+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": "waiting",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-12T03:34:24.00Z"
      }
    }
  }
}
```